Virual env: myenv

# Requirements:
- transformers=2.6.0
- torch===1.7.1+cu110
- torchvision===0.8.2+cu110
- torchaudio===0.7.2

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Importings:

In [ ]:
import ast
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import transformers
from transformers.modeling_bert import BertForTokenClassification
from transformers.optimization import AdamW
from transformers.optimization import get_linear_schedule_with_warmup
from seqeval.metrics import f1_score, accuracy_score
from tqdm import trange 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import recall_score,classification_report,f1_score, accuracy_score, precision_score


* For performance issues, a GPU is needed for our model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device('cuda:1')
n_gpu = torch.cuda.device_count()
print(n_gpu)

In [ ]:
def FindMaxLength(lst):
    maxList = max(lst, key = lambda i: len(i))
    maxLength = len(maxList)
     
    return maxLength

In [ ]:
df = pd.read_csv('./uspto_df_final.csv')
sentences = df['claim_segments'].to_list()
sentences = [ast.literal_eval(x) for x in sentences]
labels = df['claim_segments_binary'].to_list()
labels = [ast.literal_eval(x) for x in labels]

In [ ]:
tag_values = [0,1]
tag_values.append(2)
tag2idx = {t: i for i, t in enumerate(tag_values)}
n_tags = len(tag_values)
MAX_LEN = 512
bs = 16

# Description of the model
Our model is a pre-trained BERT
- Input:
X = an array of sentences where a sentence is a list of claim segments.
- Output:
y = an array of tags where a tag is a list containing binary values for each element in the sequence

- We make use of a pre-trained tokenizer included with the BERT implmentation. The tokenizer is based on a word-piece tokenizer that tokenizes the sentence to words that appear in the vocabulary.
- We create tokenize_and_preserve_labels that take as input a list of sentences and a list of labes and tokenizes them without loosing the mapping between the tokens. That means tokenized word should have his tag tokenized in the same index 

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        tokenized_sentence.extend(tokenized_word)
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]



- To get the token and label sequences to the length we want, we cut and pad them.

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx[2], padding="post",
                     dtype="long", truncating="post")

* In order to ignore the padded components in the sequences, we now create the mask.

In [ ]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]

* We split the data to train, test and validation sets. After that, we convert the voctors to tensors to prepare it for the model

In [ ]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.2)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.2)

tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)



* To define the dataloaders. We shuffle the data at training time with the RandomSampler and at test time we just pass them sequentially with the SequentialSampler.

In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

# Prepare the model:
* we are using the BERT model for classification of tokens. So the best model to use is BertForTokenClassification from transformers library. 
* We load the pre-trained bert-base-cased model and provide the number of possible labels.

In [ ]:
model = BertForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)


In [ ]:
model.cuda();

* Prepare the optimizer to be used: The optimizer must be set up, and the parameters that it should update must be added.
The AdamW optimizer is a common option. 


In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)


* We add a scheduler to linearly reduce the learning rate throughout the epochs.
* We fix the number of total steps: Total number of training steps is number of batches * number of epochs.

In [ ]:
epochs = 1
max_grad_norm = 1.0

total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


# Fit the BERT model

In [ ]:
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    #Training
    
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        loss.backward()
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    #Validation
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
            
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))


# Summary of evaluation metrics

In [ ]:
print('epochs =',epochs,'|', 'batch_size=', bs, '|', 'optimizer: ',optimizer,'|', 'loss:', loss, '\n')
print('recall_score', recall_score(valid_tags, pred_tags, average='macro', labels=np.unique(pred_tags)))
print('f1_score',f1_score(valid_tags, pred_tags, average='macro', labels=np.unique(pred_tags)))
print('accuracy_score',accuracy_score(valid_tags, pred_tags))
print('precision_score',precision_score(valid_tags, pred_tags, average='macro'))

* Plot the learning curve.

In [ ]:
%matplotlib inline

sns.set(style='darkgrid')

sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)


plt.plot(loss_values, 'b-o', label="training loss")
plt.plot(validation_loss_values, 'r-o', label="validation loss")

plt.xticks([0, 1, 2])
plt.title("Learning curve")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

plt.show()


# Apply to a new sentence

In [ ]:
test_sentence = ' '.join(ast.literal_eval(df['claim_segments'][3000]))


In [ ]:
tokenized_sentence = tokenizer.encode(test_sentence)
input_ids = torch.tensor([tokenized_sentence]).cuda()


In [ ]:
with torch.no_grad():
    output = model(input_ids)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)


In [ ]:
# join bpe split tokens
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_labels.append(tag_values[label_idx])
        new_tokens.append(token)


In [ ]:
for token, label in zip(new_tokens, new_labels):
    print("{}\t{}".format(label, token))
